In [91]:
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    StaleElementReferenceException,
    ElementClickInterceptedException
)

url = "https://www.reclameaqui.com.br/empresa/enel-distribuicao-ceara/"

chrome_options = Options()
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

driver = webdriver.Chrome(options=chrome_options)
driver.get(url)

In [92]:
#Aceitar o cookies
driver.find_element(By.LINK_TEXT, 'Aceitar todos').click()

#Acessando todas as reclamações da página
allComplaints = driver.find_element(By.ID, 'box-complaints-read-all')
allComplaints.click()

In [57]:
webScrapingELement = {
    "location": ("XPATH", "//span[@data-testid='complaint-location']"),
    "date": ("XPATH", "//span[@data-testid='complaint-creation-date']"),
    "id": ("XPATH", "//span[@data-testid='complaint-id']"),
    "category": ("CSS_SELECTOR", "li[data-testid='listitem-categoria'] a"),
    "product": ("CSS_SELECTOR", "li[data-testid='listitem-produto'] a"),
    "problem": ("CSS_SELECTOR", "li[data-testid='listitem-problema'] a"),
    "complaint": ("CLASS_NAME", 'sc-lzlu7c-17'),
    "response": ("CLASS_NAME", "sc-1a60wwz-1")
}

# Lista para armazenar os dados
list = {key: [] for key in webScrapingELement.keys()}

In [ ]:
numberOfPages = 0

while numberOfPages < 8:
    try:
        
        titleElements = driver.find_elements(By.TAG_NAME, 'h4')
        index = 0

        while index < len(titleElements):
            
            try:
                titleElements = driver.find_elements(By.TAG_NAME, 'h4')
                title = titleElements[index]
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.TAG_NAME, 'h4')))         # Esperar até que o título seja clicável
                actions = ActionChains(driver)                                                           # Scroll para o elemento se estiver fora da visão                                                   
                actions.move_to_element(title).perform()
                time.sleep(random.uniform(2, 5))
                title.click()

                for name, (method, selector) in webScrapingELement.items():
                    
                    try:
                        element_text = driver.find_element(getattr(By, method), selector).text
                    except NoSuchElementException:
                        element_text = ""
                    list[name].append(element_text)

                driver.back()
                index += 1

            except TimeoutException:
                driver.back()
                continue
            
            except ElementClickInterceptedException:
                actions = ActionChains(driver)                                                          # Scroll para o elemento se ocorrer um ElementClickInterceptedException
                actions.move_to_element(title).perform()
                title.click()
                continue

            except StaleElementReferenceException:
                titleElements = driver.find_elements(By.TAG_NAME, 'h4')                                 # Se ocorrer a exceção, atualize a lista de elementos e continue o loop
                pass
    
    except TimeoutException:
        titleElements = driver.find_elements(By.TAG_NAME, 'h4')
        continue
    
    except StaleElementReferenceException:
        pass                                                                                            # Ignora a exceção no nível mais externo do loop
    
    numberOfPages += 1 
    
    try:
        nextPage = driver.find_element(By.CSS_SELECTOR, 'button[data-testid="next-page-navigation-button"]')
        nextPage.click()
    except NoSuchElementException:
        break
    except StaleElementReferenceException:
        pass

In [94]:
df = pd.DataFrame(list)

In [95]:
df.to_excel('/Users/pesati/Desktop/webscraping/dataset1.xlsx', index=False)